Problem Set 2

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/mit_ct_2016/Assignments/PS2

/content/drive/MyDrive/mit_ct_2016/Assignments/PS2


In [6]:
# Finding shortest paths through MIT buildings

import unittest
from graph import Digraph, Node, WeightedEdge

# Problem 2: Building up the Campus Map
#

In [10]:
# Problem 2b: Implementing load_map
def load_map(map_filename):
    """
    Parses the map file and constructs a directed graph

    Parameters:
        map_filename : name of the map file

    Assumes:
        Each entry in the map file consists of the following four positive
        integers, separated by a blank space:
            From To TotalDistance DistanceOutdoors
        e.g.
            32 76 54 23
        This entry would become an edge from 32 to 76.

    Returns:
        a Digraph representing the map
    """

    # TODO
    print("Loading map from file...")
    file = open(map_filename, 'r') # open the file
    g = Digraph()
    for line in file:  # read each line of the file
        line = line.strip('\n')  # remove the \n character
        line = line.split(' ')
        for i in range(0, 2):
            nod = Node(line[i])
            if not g.has_node(nod):
                g.add_node(nod)
        wei_edge = WeightedEdge(Node(line[0]), Node(line[1]), int(line[2]), int(line[3]))
        g.add_edge(wei_edge)
    file.close()
    return g




In [18]:
#test run
print(load_map('mit_map.txt'))

Loading map from file...



In [27]:
# Problem 3: Finding the Shorest Path using Optimized Search Method

# Problem 3b: Implement get_best_path

"""
    Finds the shortest path between buildings subject to constraints.

    Parameters:
        digraph: Digraph instance
            The graph on which to carry out the search
        start: string
            Building number at which to start
        end: string
            Building number at which to end
        path: list composed of [[list of strings], int, int]
            Represents the current path of nodes being traversed. Contains
            a list of node names, total distance traveled, and total
            distance outdoors.
        max_dist_outdoors: int
            Maximum distance spent outdoors on a path
        best_dist: int
            The smallest distance between the original start and end node
            for the initial problem that you are trying to solve
        best_path: list of strings
            The shortest path found so far between the original start
            and end node.

    Returns:
        A tuple with the shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k and the distance of that path.

        If there exists no path that satisfies max_total_dist and
        max_dist_outdoors constraints, then return None.
"""
    # TODO
def getDistance(digraph, path):
    total_dist = 0
    outdoor_dist = 0
    for i in range(len(path) - 1):
        for edge in digraph.edges[path[i]]:
            if edge.dest == path[i + 1]:
                total_dist += edge.get_total_distance()
                outdoor_dist += edge.get_outdoor_distance()
    return (total_dist, outdoor_dist)

def get_best_path(digraph, start, end, path, max_dist_outdoors, best_dist,
                  best_path):
  path = path + [start]
  if start not in digraph.nodes:
        raise ValueError('Invalid Node')
  elif start == end:
        return path
  else:
        for node in digraph.get_edges_for_node(start):
            if node.dest not in path:
                    new_path = get_best_path(digraph, node.dest, end, path, max_dist_outdoors, best_dist, best_path)
                    if new_path != None:
                        total_dist, outdoor_dist = getDistance(digraph, new_path)
                        if outdoor_dist <= max_dist_outdoors and total_dist <= best_dist:
                            best_path = new_path
                            best_dist = total_dist
  return best_path

In [29]:
# Problem 3c: Implement directed_dfs
def directed_dfs(digraph, start, end, max_total_dist, max_dist_outdoors):
    """
    Finds the shortest path from start to end using a directed depth-first
    search. The total distance traveled on the path must not
    exceed max_total_dist, and the distance spent outdoors on this path must
    not exceed max_dist_outdoors.

    Parameters:
        digraph: Digraph instance
            The graph on which to carry out the search
        start: string
            Building number at which to start
        end: string
            Building number at which to end
        max_total_dist: int
            Maximum total distance on a path
        max_dist_outdoors: int
            Maximum distance spent outdoors on a path

    Returns:
        The shortest-path from start to end, represented by
        a list of building numbers (in strings), [n_1, n_2, ..., n_k],
        where there exists an edge from n_i to n_(i+1) in digraph,
        for all 1 <= i < k

        If there exists no path that satisfies max_total_dist and
        max_dist_outdoors constraints, then raises a ValueError.
    """
    # TODO

    start = Node(start)
    end = Node(end)
    best_path = get_best_path(digraph, start, end, [], max_dist_outdoors, max_total_dist, None)

    if best_path == None:
        raise ValueError
    return [node.get_name() for node in best_path]

In [33]:
# ================================================================
# Begin tests -- you do not need to modify anything below this line
# ================================================================

class Ps2Test(unittest.TestCase):
    LARGE_DIST = 99999

    def setUp(self):
        self.graph = load_map("/content/drive/MyDrive/mit_ct_2016/Assignments/PS2/mit_map.txt")

    def test_load_map_basic(self):
        self.assertTrue(isinstance(self.graph, Digraph))
        self.assertEqual(len(self.graph.nodes), 37)
        all_edges = []
        for _, edges in self.graph.edges.items():
            all_edges += edges  # edges must be dict of node -> list of edges
        all_edges = set(all_edges)
        self.assertEqual(len(all_edges), 129)

    def _print_path_description(self, start, end, total_dist, outdoor_dist):
        constraint = ""
        if outdoor_dist != Ps2Test.LARGE_DIST:
            constraint = "without walking more than {}m outdoors".format(
                outdoor_dist)
        if total_dist != Ps2Test.LARGE_DIST:
            if constraint:
                constraint += ' or {}m total'.format(total_dist)
            else:
                constraint = "without walking more than {}m total".format(
                    total_dist)

        print("------------------------")
        print("Shortest path from Building {} to {} {}".format(
            start, end, constraint))

    def _test_path(self,
                   expectedPath,
                   total_dist=LARGE_DIST,
                   outdoor_dist=LARGE_DIST):
        start, end = expectedPath[0], expectedPath[-1]
        self._print_path_description(start, end, total_dist, outdoor_dist)
        dfsPath = directed_dfs(self.graph, start, end, total_dist, outdoor_dist)
        print("Expected: ", expectedPath)
        print("DFS: ", dfsPath)
        self.assertEqual(expectedPath, dfsPath)

    def _test_impossible_path(self,
                              start,
                              end,
                              total_dist=LARGE_DIST,
                              outdoor_dist=LARGE_DIST):
        self._print_path_description(start, end, total_dist, outdoor_dist)
        with self.assertRaises(ValueError):
            directed_dfs(self.graph, start, end, total_dist, outdoor_dist)

    def test_path_one_step(self):
        self._test_path(expectedPath=['32', '56'])

    def test_path_no_outdoors(self):
        self._test_path(
            expectedPath=['32', '36', '26', '16', '56'], outdoor_dist=0)

    def test_path_multi_step(self):
        self._test_path(expectedPath=['2', '3', '7', '9'])

    def test_path_multi_step_no_outdoors(self):
        self._test_path(
            expectedPath=['2', '4', '10', '13', '9'], outdoor_dist=0)

    def test_path_multi_step2(self):
        self._test_path(expectedPath=['1', '4', '12', '32'])

    def test_path_multi_step_no_outdoors2(self):
        self._test_path(
            expectedPath=['1', '3', '10', '4', '12', '24', '34', '36', '32'],
            outdoor_dist=0)

    def test_impossible_path1(self):
        self._test_impossible_path('8', '50', outdoor_dist=0)

    def test_impossible_path2(self):
        self._test_impossible_path('10', '32', total_dist=100)


if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


..FEEEEEE

Loading map from file...
------------------------
Shortest path from Building 8 to 50 without walking more than 0m outdoors
Loading map from file...
------------------------
Shortest path from Building 10 to 32 without walking more than 100m total
Loading map from file...
Loading map from file...
------------------------
Shortest path from Building 2 to 9 
Loading map from file...
------------------------
Shortest path from Building 1 to 32 
Loading map from file...
------------------------
Shortest path from Building 2 to 9 without walking more than 0m outdoors
Loading map from file...
------------------------
Shortest path from Building 1 to 32 without walking more than 0m outdoors
Loading map from file...
------------------------
Shortest path from Building 32 to 56 without walking more than 0m outdoors
Loading map from file...
------------------------
Shortest path from Building 32 to 56 



ERROR: test_path_multi_step (__main__.Ps2Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-33-dec8c1ebfb21>", line 64, in test_path_multi_step
    self._test_path(expectedPath=['2', '3', '7', '9'])
  File "<ipython-input-33-dec8c1ebfb21>", line 42, in _test_path
    dfsPath = directed_dfs(self.graph, start, end, total_dist, outdoor_dist)
  File "<ipython-input-29-e7357d03b1e3>", line 34, in directed_dfs
    best_path = get_best_path(digraph, start, end, [], max_dist_outdoors, max_total_dist, None)
  File "<ipython-input-27-c1d518f13079>", line 53, in get_best_path
    raise ValueError('Invalid Node')
ValueError: Invalid Node

ERROR: test_path_multi_step2 (__main__.Ps2Test)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-33-dec8c1ebfb21>", line 71, in test_path_multi_step2
    self._test_path(expectedPath=['1', '4', '12', 